In [4]:
import torch
import torch.nn as nn
import torch.functional as F

In [13]:
# Transformer block implementation

'''
Input: Input embedding vector
Parameters:
Output:
Note - dimensions of input is preserved after the trasnformre block operations.
'''
GPT_CFG_124M = {
    'vocab_size': 50257,
    'emb_dim': 768,
    'context_length': 1024,
    'n_heads': 12,
    'n_layers': 12,
    'drop_rate': 0.1,
    'qkv_bias': False
}

In [18]:
class MultiHeadAttention(nn.Module):
  def __init__(self,d_in,d_out,context_length,dropout,num_heads,qkv_bias=False): #included a num_heads parameter
    super().__init__()
    assert (d_out%num_heads==0), \
    "d_out must be divisible by num_heads"

    self.d_out = d_out
    self.num_heads = num_heads
    self.head_dim = d_out//num_heads

    self.W_query = nn.Linear(d_in,d_out,bias=qkv_bias)
    self.W_key = nn.Linear(d_in,d_out,bias=qkv_bias)
    self.W_value = nn.Linear(d_in,d_out,bias=qkv_bias)
    self.output_proj = nn.Linear(d_out,d_out) # linear layer to combine the outputs from the different attention heads

    self.dropout = nn.Dropout(dropout)
    mask = torch.triu(torch.ones(context_length,context_length),diagonal=1) # causal attention mask which makes all elements in upper triangle '1'
    self.register_buffer('mask',mask)


  def forward(self,X):
    b,num_tokens,d_in = X.shape # new batch dimesnion b

    queries = self.W_query(X) # shape (b,num_tokens,d_out)
    keys = self.W_key(X) # shape (b,num_tokens,d_out)
    values = self.W_value(X) # shape (b,num_tokens,d_out)

    # Implicitly split the matrix by adding a 'num_heads' dimension
    # Unroll last dimension: (b,num_tokens,d_out) -> (b,num_heads,num_heads,head_dim)

    queries = queries.view(b,num_tokens,self.num_heads,self.head_dim)
    keys = keys.view(b,num_tokens,self.num_heads,self.head_dim)
    values = values.view(b,num_tokens,self.num_heads,self.head_dim)

    # Transpose: We need to group by the number of heads instead of the number of tokens
    # We do a transopose to achieve this: (b,num_tokens,num_heads,head_dim) -> (b,,num_heads,num_tokens,head_dim)

    queries = queries.transpose(1,2)
    keys = keys.transpose(1,2)
    values = values.transpose(1,2)

    attn_scores = queries@keys.transpose(2,3) # initial keys shape: (b,num_heads,num_tokens,head_dim) after transposing last two dim, new shape: (b,num_heads,head_dim,num_tokens),
    # attn_scores.shape: (b, d_out,num_tokens, num_tokens)
    attn_scores.masked_fill_(
        self.mask.bool()[:num_tokens,:num_tokens],-torch.inf
    ) # apply mask only for that particular sequence length/num tokens

    attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5,dim=-1) # normalize by sqrt(d_out of keys) because it helps reduce variance of softmax
    attn_weights = self.dropout(attn_weights) # dropout layer -  drops %dropout from each layer. Used to prevent overfitting.

    context_vector = (attn_weights@values).transpose(1,2) # group by tokens for each head so its easier the merge, shape: (b,num_tokens,num_heads,head_dim)
    # combine heads, self.d_out = num_heads*head_dim
    context_vector = context_vector.contiguous().view(b,num_tokens,self.d_out) #contigous makes the matrix 'C contiguous' meaning that rows are stored next to each other in memory
    # link - 'https://stackoverflow.com/questions/48915810/what-does-contiguous-do-in-pytorch' - notes on contigous
    context_vector = self.output_proj(context_vector) #optional projection

    return context_vector

In [8]:
class LayerNormalization(nn.Module):
  def __init__(self,emb_dim):
    super().__init__()
    self.eps = 1e-5 # epsilon - used to avoid dividing by 0
    self.scale = nn.Parameter(torch.ones(emb_dim)) # scale - trainable param
    self.shift = nn.Parameter(torch.zeros(emb_dim)) # shfit - trainable param

  def forward(self,x):
    mean = x.mean(dim=-1,keepdims=True)
    var = x.var(dim=-1,keepdims=True,unbiased=False) # unbiased is for bessel's correction, True uses bessel's correction
    x_norm = (x-mean)/torch.sqrt(var+self.eps) # standardization by adding epsilon
    return self.scale*x_norm + self.shift

In [9]:
class GELU(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self,x):
    return 0.5*x*(1+torch.tanh(torch.sqrt(torch.tensor(2.0/torch.pi)) *
                               (x+0.044715*torch.pow(x,3))
                               )) # phi(x) is approximated using this formula

In [10]:
class FeedForward(nn.Module):
  def __init__(self,cfg):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(cfg['emb_dim'],4*cfg['emb_dim']),
        GELU(),
        nn.Linear(4*cfg['emb_dim'],cfg['emb_dim'])
    )

  def forward(self,x):
    return self.layers(x)

In [11]:
class SkipConnections(nn.Module):
  def __init__(self,layers_dim,use_shortcut):
    super().__init__()
    self.use_shortcut = use_shortcut
    # Create Linear layers based on provided dimensions in layers_dim
    self.layers = nn.ModuleList([
        nn.Sequential(
            nn.Linear(layers_dim[0], layers_dim[1]),
            GELU(),
        ),
        nn.Sequential(
            nn.Linear(layers_dim[1], layers_dim[0]),
            GELU(),
        ),
        nn.Sequential(
            nn.Linear(layers_dim[2], layers_dim[3]),
            GELU(),
        ),
        nn.Sequential(
            nn.Linear(layers_dim[3], layers_dim[4]),
            GELU(),
        ),
        nn.Sequential(
            nn.Linear(layers_dim[4], layers_dim[5]),
            GELU(),
        ),
    ])

  def forward(self,x):
    for layer in self.layers:
      # calculate output
      out = layer(x)
      # check if shortcut can be applied
      if self.use_shortcut and out.shape==x.shape:
        x = x + out
      # use if not shortcut can be applied
      else:
        x = out

    return x


In [12]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNormalization(cfg["emb_dim"])
        self.norm2 = LayerNormalization(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

In [16]:
# testing with a sample input
torch.manual_seed(123)
x = torch.rand(2, 4, 768) # [2 batches, 4 tokens per batch, 768 embedding dimension per token]
block = TransformerBlock(GPT_CFG_124M)
output = block(x)
print("Input shape:", x.shape)
print("Output shape:", output.shape)

Input shape: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768])
